In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import load_model
from datetime import date
from keras.utils import np_utils
import datetime
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from keras.layers import BatchNormalization
import time

%matplotlib inline

/Users/kaushalalate/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv("df.csv", encoding="ISO-8859-1")
df = df.reset_index()

In [3]:
def visualise_null(data):
    sns.set(font_scale=1.3)
    fig, ax = plt.subplots(figsize=(8,5))
    sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')
    plt.xticks(range(len(data.columns)), data.columns, rotation=60)
    plt.show()

In [4]:
# visualise_null(df)

In [5]:
df.shape

(63207, 103)

In [6]:
seed = 23
np.random.seed(seed)

In [7]:
df = pd.get_dummies(df, columns=['grade'])

In [8]:
col_y = ['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G']

In [9]:
df = df.drop(["title", "emp_title", "desc"], axis=1)
df_Y = df[col_y]
df_X = df.drop(col_y, axis=1)

In [10]:
df_X = df_X.drop('index', axis=1)

In [11]:
df_X.shape, df_Y.shape, df.shape

((63207, 98), (63207, 7), (63207, 106))

In [12]:
# desc_vectors = pd.read_csv("custom_desc_vectors.csv")
# title_vectors = pd.read_csv("custom_title_vectors.csv")
# emp_title_vectors = pd.read_csv("custom_emp_title_vectors.csv")

In [13]:
# df_X = pd.concat([df_X, desc_vectors, title_vectors, emp_title_vectors], axis=1)

In [14]:
print(df_X.shape)

(63207, 98)


In [23]:
def model():
    # create model
    model = Sequential()
    model.add(Dense(5, input_dim=98, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
estimator = KerasClassifier(build_fn=model, epochs=30, batch_size=1024, verbose=0)

In [25]:
kfold = KFold(n_splits=2, shuffle=True, random_state=seed)

In [26]:
start = time.clock()
results = cross_val_score(estimator, df_X, df_Y, cv=kfold)
print(time.clock() - start)

29.110516000000004


In [27]:
print(results.mean())

0.4903887326178669


In [28]:
print("Model with embeddings: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model with embeddings: 49.04% (0.06%)
